## Open, Plot and Explore Lidar Data in Raster Format with Python

### What is a Raster?
Raster or “gridded” data are stored as a grid of values which are rendered on a map as pixels. Each pixel value represents an area on the Earth’s surface. A raster file is a composed of regular grid of cells, all of which are the same size. You’ve looked at and used rasters before if you’ve looked at photographs or imagery in a tool like Google Earth. However, the raster files that you will work with are different from photographs in that they are spatially referenced. Each pixel represents an area of land on the ground. That area is defined by the spatial resolution of the raster.

<img src="images/raster-concept.png" alt="drawing" width="800"/>

## Raster Facts

A few notes about rasters:

* Each cell is called a pixel.

* And each pixel represents an area on the ground.

* The resolution of the raster represents the area that each pixel represents the area it represents on the ground. So, a 1 meter resolution raster, means that each pixel represents a 1 m by 1m area on the ground.

A raster dataset can have attributes associated with it as well. For instance in a LiDAR derived digital elevation model (DEM), each cell represents an elevation value for that location on the earth. In a LIDAR derived intensity image, each cell represents a LIDAR intensity value or the amount of light energy returned to and recorded by the sensor.

<img src="images/raster-resolution.png" alt="drawing" width="1000"/>

## Open Raster Data in Python

You can use the `rasterio` library combined with `numpy` and `matplotlib` to open, manipulate and plot raster data in Python. To begin you will load a suite of python libraries required to complete this lesson. These libraries are all a part of the `uq-geo` conda environment.

In [ ]:
import os
import geopandas as gpd
from geopandas.tools import sjoin
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
import rasterio as rio
from rasterio.plot import show
from rasterstats import zonal_stats
import rasterstats
import osmnx as ox
import contextily as cx
import pandas as pd

Be sure to set your working directory `os.chdir("path-to-you-dir-here/uq-geo-python/data")`

In [ ]:
os.chdir(os.path.join('/home/dev103/uq-geo-python'))

height_raster_path = 'data/GEDI_LandSat_Raster/Forest_Height_2019.tif'

# open raster data
height_raster = rio.open(height_raster_path)

# view spatial extent
height_raster.bounds

In [ ]:
# plot the raster using matplotlib

fig, ax = plt.subplots(figsize = (10,10))
show(height_raster, 
     title="Canopy Height 2019", 
     ax=ax)
ax.set_axis_off()

### Opening and Closing File Connections
The rasterio library is efficient as it establishes a connection with the raster file rather than directly reading it into memory. Because it creates a connection, it is important that you close the connection after it is opened AND after you’ve finished working with the data!

In [ ]:
# close the file connection
height_raster.close()

In [ ]:
# this returns an error as you have closed the connection to the file. 
show(height_raster)

Once the connection is closed, you can no longer work with the data. You’ll need to re-open the connection. Like this:

In [ ]:
# open raster data connection - again
height_raster = rio.open(height_raster_path)

fig, ax = plt.subplots(figsize = (10,10))

# You can use the rasterio show() function to quickly plot a raster image.
show(height_raster, 
     title="Once the connection is re-opened \nyou can work with the raster data", 
     ax=ax)
ax.set_axis_off()

In [ ]:
height_raster.close()

### Context Manager to Open/Close Raster Data

A better way to work with raster data in `rasterio` is to use the context manager. This will handle opening and closing the raster file for you.

`with rio.open('name of file') as scr: src.rasteriofunctionname()`

In [ ]:
# view spatial extent of raster object
with rio.open(height_raster_path) as src:
    print(src.bounds)

Once you are outside of the `with` statement, you can no long access the `src` object which contains the spatial raster information.

### Zonal statistics
Quite often you have a situtation when you want to summarize raster datasets based on vector geometries. Rasterstats is a Python module that helps to acheive this easily when using geopandas and rasterio.

In [ ]:
# Read in Forest height raster with rasterio.

height_raster_path = 'data/GEDI_LandSat_Raster/Forest_Height_2019.tif'

height_raster = rio.open(height_raster_path)

In [ ]:
# Read in regional ecosystem polygon as geopandas dataframe. 

RE_df_path = "data/RE_Polygons/RE_Polygons.shp"

RE_df = gpd.read_file(RE_df_path)

In [ ]:
RE_df

In [ ]:
# Plot the raster and the vector data together.
fig, ax = plt.subplots(figsize = (15,15))

# Make a numpy array to get min and max pixel values for colour bar
height_array = height_raster.read(1)

# Nomalise colour to min and max values
norm = colors.Normalize(vmin=height_array.min(), vmax=height_array.max())

# Plot raster
show(height_raster, ax = ax, cmap='viridis')
fig.colorbar(cm.ScalarMappable(norm=norm, cmap='viridis_r'), ax=ax)

# Plot vector data
RE_df.plot(ax = ax, facecolor = 'None', edgecolor = 'red')
plt.show()

In [ ]:
RE_df_disolved = RE_df.dissolve(by='RE', aggfunc='sum', as_index = False)

In [ ]:
RE_df_disolved

In [ ]:
# Assign raster values to a numpy nd array
height_array = height_raster.read(1)

# Calculate the zonal statistics 
height_raster_zonal = rasterstats.zonal_stats(RE_df_disolved, height_array, affine = height_raster.transform,
                                      stats = ['mean','std','max','min'], 
                                      geojson_out = True)

### Now that we have the zonal statistics we can load them into a pandas dataframe, which is nicer to work with. 

In [ ]:
# Create an empty list to store values
height_stats = []

# Initalise counter for loop
i = 0

# Loop through zonal statistics array and append values to list
while i < len(height_raster_zonal):
    height_stats.append(height_raster_zonal[i]['properties'])
    i = i + 1 

# Transfer the infromation from the list to a pandas dataframe
height_stats_df = pd.DataFrame(height_stats)

In [ ]:
height_stats_df

In [ ]:
# Initialise fig, ax and set figure size 
fig, ax = plt.subplots(figsize = (20, 5))

height_stats_df.plot.bar(x='RE', ax=ax)
plt.xticks(rotation=90)
plt.show()